In [1]:
import numpy as np
import pandas as pd

df_regular = pd.read_csv('DataFiles/RegularSeasonCompactResults.csv')
df_regular.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [2]:
df_team_conference = pd.read_csv('DataFiles/TeamConferences.csv')
df_team_conference.head()

,Season,TeamID,ConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


We want to compute these features from regular season. Regular season includes conference tourney. We may want to compute the same features from conference tourney seperately as they'are good proxy of NCAA tourney.
1. Each team's winning percentage
2. Each team's average winning margin
3. Each conference's winning percentage
4. Each conference's average winning margin

In [8]:
df_win_team_conference = df_team_conference.rename(columns={'TeamID':'WTeamID', 'ConfAbbrev':'WTeamConfAbbrev'})
df_win_team_conference.head()

,Season,WTeamID,WTeamConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [9]:
df_loss_team_conference = df_team_conference.rename(columns={'TeamID':'LTeamID', 'ConfAbbrev':'LTeamConfAbbrev'})
df_loss_team_conference.head()

,Season,LTeamID,LTeamConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [6]:
df_regular.drop(labels=['DayNum', 'WLoc', 'NumOT'], axis=1, inplace=True)
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore
0,1985,1228,81,1328,64
1,1985,1106,77,1354,70
2,1985,1112,63,1223,56
3,1985,1165,70,1432,54
4,1985,1192,86,1447,74


In [10]:
df = pd.merge(left=df_regular, right=df_win_team_conference, how='left', on=['Season', 'WTeamID'])
df.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev
0,1985,1228,81,1328,64,big_ten
1,1985,1106,77,1354,70,swac
2,1985,1112,63,1223,56,pac_ten
3,1985,1165,70,1432,54,ivy
4,1985,1192,86,1447,74,ecacm


In [11]:
df_regular = pd.merge(left=df, right=df_loss_team_conference, how='left', on=['Season', 'LTeamID'])
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev
0,1985,1228,81,1328,64,big_ten,big_eight
1,1985,1106,77,1354,70,swac,meac
2,1985,1112,63,1223,56,pac_ten,a_sun
3,1985,1165,70,1432,54,ivy,ind
4,1985,1192,86,1447,74,ecacm,ecacm


### Now ```df_regular``` has everything we need. We will first compute each team's winning percentage. ###

In [14]:
grouped_by_Season_WTeamID = df_regular.groupby(['Season', 'WTeamID'])
grouped_by_Season_WTeamID.LTeamID.count()

Season  WTeamID
1985    1102        5
        1103        9
        1104       21
        1106       10
        1108       19
        1109        1
        1110        7
        1111       10
        1112       18
        1113       11
        1114       17
        1116       21
        1117       11
        1119       15
        1120       18
        1121        7
        1122        6
        1123       13
        1124        7
        1126        6
        1129       12
        1130       16
        1131       14
        1132       11
        1133       16
        1134       11
        1135        8
        1137       15
        1139       16
        1140       15
                   ..
2017    1433       26
        1434       23
        1435       19
        1436       28
        1437       31
        1438       22
        1439       22
        1440        4
        1441        7
        1442        6
        1443       15
        1444       15
        1447       15
        1448    

In [15]:
df_regular_win_team = grouped_by_Season_WTeamID.LTeamID.count().reset_index(name='W').rename(columns={'WTeamID':'TeamID'})
df_regular_win_team.head()

,Season,TeamID,W
0,1985,1102,5
1,1985,1103,9
2,1985,1104,21
3,1985,1106,10
4,1985,1108,19


In [16]:
grouped_by_Season_LTeamID = df_regular.groupby(['Season', 'LTeamID'])
grouped_by_Season_LTeamID.LTeamID.count()

Season  LTeamID
1985    1102       19
        1103       14
        1104        9
        1106       14
        1108        6
        1109       23
        1110       18
        1111       14
        1112        9
        1113       16
        1114       13
        1116       12
        1117       14
        1119       13
        1120       11
        1121       19
        1122       17
        1123       16
        1124       17
        1126       19
        1129       13
        1130       10
        1131       14
        1132       15
        1133       12
        1134       13
        1135       18
        1137       10
        1139        8
        1140       14
                   ..
2017    1433        8
        1434        8
        1435       15
        1436        5
        1437        3
        1438       10
        1439       10
        1440       24
        1441       23
        1442       20
        1443       17
        1444       16
        1447       14
        1448    

In [17]:
df_regular_loss_team = grouped_by_Season_LTeamID.WTeamID.count().reset_index(name='L').rename(columns={'LTeamID':'TeamID'})
df_regular_loss_team.head()

,Season,TeamID,L
0,1985,1102,19
1,1985,1103,14
2,1985,1104,9
3,1985,1106,14
4,1985,1108,6


In [19]:
df_regular_winning_pct_team = pd.merge(left=df_regular_win_team,\
                                       right=df_regular_loss_team,\
                                       how='outer', on=['Season', 'TeamID'])
df_regular_winning_pct_team.head()

,Season,TeamID,W,L
0,1985,1102,5.0,19.0
1,1985,1103,9.0,14.0
2,1985,1104,21.0,9.0
3,1985,1106,10.0,14.0
4,1985,1108,19.0,6.0


In [20]:
df_regular_winning_pct_team.fillna(value=0, inplace=True)

In [21]:
df_regular_winning_pct_team['Pct'] = df_regular_winning_pct_team.W / (df_regular_winning_pct_team.W + df_regular_winning_pct_team.L)
df_regular_winning_pct_team.head()

,Season,TeamID,W,L,Pct
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


### Next, let's compute each team's average winning margin. ###

For the moment, we accmulate the score difference of wins and losses repectively. Along with wins and losses, we have various ways to compute average winning margin.

In [24]:
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev
0,1985,1228,81,1328,64,big_ten,big_eight
1,1985,1106,77,1354,70,swac,meac
2,1985,1112,63,1223,56,pac_ten,a_sun
3,1985,1165,70,1432,54,ivy,ind
4,1985,1192,86,1447,74,ecacm,ecacm


In [25]:
df_regular['ScoreDiff'] = df_regular.WScore - df_regular.LScore
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev,ScoreDiff
0,1985,1228,81,1328,64,big_ten,big_eight,17
1,1985,1106,77,1354,70,swac,meac,7
2,1985,1112,63,1223,56,pac_ten,a_sun,7
3,1985,1165,70,1432,54,ivy,ind,16
4,1985,1192,86,1447,74,ecacm,ecacm,12


In [33]:
grouped_by_Season_WTeamID = df_regular.groupby(['Season', 'WTeamID'])
grouped_by_Season_WTeamID.ScoreDiff.agg(np.sum)

Season  WTeamID
1985    1102        50
        1103        68
        1104       277
        1106        95
        1108       263
        1109         1
        1110        62
        1111        74
        1112       225
        1113        79
        1114       193
        1116       217
        1117        84
        1119        89
        1120       213
        1121        53
        1122        26
        1123       187
        1124        93
        1126        20
        1129       149
        1130       199
        1131       179
        1132        79
        1133       143
        1134       132
        1135        58
        1137       148
        1139       116
        1140       129
                  ... 
2017    1433       342
        1434       301
        1435       252
        1436       403
        1437       524
        1438       435
        1439       267
        1440        45
        1441        49
        1442        53
        1443       150
        1444      

In [34]:
df_regular_win_team = grouped_by_Season_WTeamID.ScoreDiff.agg(np.sum).reset_index(name='WAccumScore').rename(columns={'WTeamID':'TeamID'})
df_regular_win_team.head()

,Season,TeamID,WAccumScore
0,1985,1102,50
1,1985,1103,68
2,1985,1104,277
3,1985,1106,95
4,1985,1108,263


In [35]:
grouped_by_Season_LTeamID = df_regular.groupby(['Season', 'LTeamID'])
grouped_by_Season_LTeamID.ScoreDiff.apply(np.sum)

Season  LTeamID
1985    1102       189
        1103       138
        1104        43
        1106       186
        1108        64
        1109       700
        1110       256
        1111       141
        1112        31
        1113       189
        1114       129
        1116        97
        1117       156
        1119       100
        1120       106
        1121       365
        1122       223
        1123       139
        1124       203
        1126       339
        1129       114
        1130        62
        1131       125
        1132       145
        1133       100
        1134       149
        1135       133
        1137        91
        1139        91
        1140       113
                  ... 
2017    1433        61
        1434        99
        1435       146
        1436        55
        1437        18
        1438        84
        1439       112
        1440       384
        1441       487
        1442       277
        1443       243
        1444      

In [36]:
df_regular_loss_team = grouped_by_Season_LTeamID.ScoreDiff.apply(np.sum).reset_index(name='LAccumScore').rename(columns={'LTeamID':'TeamID'})
df_regular_loss_team.head()

,Season,TeamID,LAccumScore
0,1985,1102,189
1,1985,1103,138
2,1985,1104,43
3,1985,1106,186
4,1985,1108,64


In [37]:
df_regular_winning_margin_team = pd.merge(left=df_regular_win_team,\
                                          right=df_regular_loss_team,\
                                          how='outer', on=['Season', 'TeamID'])
df_regular_winning_margin_team.head()

,Season,TeamID,WAccumScore,LAccumScore
0,1985,1102,50.0,189.0
1,1985,1103,68.0,138.0
2,1985,1104,277.0,43.0
3,1985,1106,95.0,186.0
4,1985,1108,263.0,64.0


In [38]:
df_regular_winning_margin_team.fillna(value=0, inplace=True)

In [41]:
df_regular_results_team = pd.merge(left=df_regular_winning_margin_team,\
                                   right=df_regular_winning_pct_team,\
                                   on=['Season', 'TeamID'],\
                                   how='inner')
df_regular_results_team.head()

,Season,TeamID,WAccumScore,LAccumScore,W,L,Pct
0,1985,1102,50.0,189.0,5.0,19.0,0.208333
1,1985,1103,68.0,138.0,9.0,14.0,0.391304
2,1985,1104,277.0,43.0,21.0,9.0,0.700000
3,1985,1106,95.0,186.0,10.0,14.0,0.416667
4,1985,1108,263.0,64.0,19.0,6.0,0.760000


### Cells below are initial commit. ###

In [27]:
df_conference_tourney[df_conference_tourney.ConfAbbrev=='ivy']

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
4406,2017,ivy,131,1343,1335
4407,2017,ivy,131,1463,1217
4408,2017,ivy,132,1343,1463


In [28]:
df_regular_clean.head()

,Season,WTeamID,LTeamID
0,1985,1228,1328
1,1985,1106,1354
2,1985,1112,1223
3,1985,1165,1432
4,1985,1192,1447


In [35]:
grouped_win = df_regular_clean.groupby(['Season', 'WTeamID'])
grouped_loss = df_regular_clean.groupby(['Season', 'LTeamID'])

In [48]:
win = grouped_win.LTeamID.count().reset_index(name='W').rename(columns={'WTeamID':'TeamID'})
win.head()
len(win)

10523

In [49]:
loss = grouped_loss.WTeamID.count().reset_index(name='L').rename(columns={'LTeamID':'TeamID'})
loss.head()
len(loss)

10534

In [57]:
df_regular_win_rate = pd.merge(left=win, right=loss, how='outer', on=['Season', 'TeamID'])
df_regular_win_rate.head()

,Season,TeamID,W,L
0,1985,1102,5.0,19.0
1,1985,1103,9.0,14.0
2,1985,1104,21.0,9.0
3,1985,1106,10.0,14.0
4,1985,1108,19.0,6.0


In [59]:
df_regular_win_rate['WRate'] = df_regular_win_rate.W / (df_regular_win_rate.W + df_regular_win_rate.L)

In [60]:
df_regular_win_rate.head()

,Season,TeamID,W,L,WRate
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


In [62]:
df_tourney.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [63]:
df_tourney_clean = df_tourney.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
df_tourney_clean.head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [64]:
df_WTeam_WRate = df_regular_win_rate.rename(columns={'TeamID':'WTeamID', 'WRate':'WTeamWRate'})
df_WTeam_WRate.head()

,Season,WTeamID,W,L,WTeamWRate
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


In [65]:
df_LTeam_WRate = df_regular_win_rate.rename(columns={'TeamID':'LTeamID', 'WRate':'LTeamWRate'})
df_LTeam_WRate.head()

,Season,LTeamID,W,L,LTeamWRate
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


In [66]:
df_WTeam_WRate.drop(labels=['W', 'L'], axis=1, inplace=True)
df_WTeam_WRate.head()

,Season,WTeamID,WTeamWRate
0,1985,1102,0.208333
1,1985,1103,0.391304
2,1985,1104,0.700000
3,1985,1106,0.416667
4,1985,1108,0.760000


In [67]:
df_LTeam_WRate.drop(labels=['W', 'L'], axis=1, inplace=True)
df_LTeam_WRate.head()

,Season,LTeamID,LTeamWRate
0,1985,1102,0.208333
1,1985,1103,0.391304
2,1985,1104,0.700000
3,1985,1106,0.416667
4,1985,1108,0.760000


In [68]:
df = pd.merge(left=df_tourney_clean, right=df_WTeam_WRate, how='left', on=['Season', 'WTeamID'])
df.head()

,Season,WTeamID,LTeamID,WTeamWRate
0,1985,1116,1234,0.636364
1,1985,1120,1345,0.620690
2,1985,1207,1250,0.925926
3,1985,1229,1425,0.740741
4,1985,1242,1325,0.766667


In [69]:
df_tourney_with_regular_win_rate = pd.merge(left=df, right=df_LTeam_WRate, how='left', on=['Season', 'LTeamID'])
df_tourney_with_regular_win_rate.head()

,Season,WTeamID,LTeamID,WTeamWRate,LTeamWRate
0,1985,1116,1234,0.636364,0.666667
1,1985,1120,1345,0.620690,0.680000
2,1985,1207,1250,0.925926,0.379310
3,1985,1229,1425,0.740741,0.678571
4,1985,1242,1325,0.766667,0.740741


In [70]:
df_tourney_with_regular_win_rate['WRateDiff'] = df_tourney_with_regular_win_rate.WTeamWRate - df_tourney_with_regular_win_rate.LTeamWRate
df_tourney_with_regular_win_rate.head()

,Season,WTeamID,LTeamID,WTeamWRate,LTeamWRate,WRateDiff
0,1985,1116,1234,0.636364,0.666667,-0.030303
1,1985,1120,1345,0.620690,0.680000,-0.059310
2,1985,1207,1250,0.925926,0.379310,0.546616
3,1985,1229,1425,0.740741,0.678571,0.062169
4,1985,1242,1325,0.766667,0.740741,0.025926


In [71]:
df_wins = pd.DataFrame()
df_wins['WRateDiff'] = df_tourney_with_regular_win_rate['WRateDiff']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['WRateDiff'] = -df_tourney_with_regular_win_rate['WRateDiff']
df_losses['Result'] = 0

df_predictions = pd.concat([df_wins, df_losses])
df_predictions.head()

,WRateDiff,Result
0,-0.030303,1
1,-0.059310,1
2,0.546616,1
3,0.062169,1
4,0.025926,1


In [74]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [75]:
X_train = df_predictions.WRateDiff.values.reshape(-1,1)
y_train = df_predictions.Result.values
X_train, y_train = shuffle(X_train, y_train)

In [77]:
np.any(np.isnan(X_train))

True

In [78]:
df_predictions[np.isnan(df_predictions.WRateDiff)]

,WRateDiff,Result
406,NaN,1
424,NaN,1
429,NaN,1
435,NaN,1
438,NaN,1
1884,NaN,1
1896,NaN,1
1926,NaN,1
1953,NaN,1
1969,NaN,1


In [79]:
df = df_predictions.dropna(axis=0, how='any')
df[np.isnan(df.WRateDiff)]

,WRateDiff,Result


In [80]:
df.head()

,WRateDiff,Result
0,-0.030303,1
1,-0.059310,1
2,0.546616,1
3,0.062169,1
4,0.025926,1


In [82]:
df_predictions.dropna(axis=0, how='any', inplace=True)
X_train = df_predictions.WRateDiff.values.reshape(-1,1)
y_train = df_predictions.Result.values
X_train, y_train = shuffle(X_train, y_train)

In [83]:
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

Best log_loss: -0.6339, with best C: 100.0


In [1]:
X = np.linspace(-1, 1, num=100).reshape(-1, 1)
preds = clf.predict_proba(X)[:,1]

plt.plot(X, preds)
plt.xlabel('Team1 WRate - Team2 WRate')
plt.ylabel('P(Team1 will win)')
plt.show()

NameError: name 'np' is not defined